In [1]:
from datetime import datetime
import math
import time
import tensorflow as tf

In [2]:
batch_size = 32
num_batches = 100

In [3]:
def print_activations(t):
    print(t.op.name, ' ', t.get_shape().as_list())

In [8]:
def inference(images):
    parameters = []
    
    with tf.name_scope('conv1') as scope:
        kernel = tf.Variable(tf.truncated_normal([11, 11, 3, 64],
                                                dtype = tf.float32, stddev = 1e-1), name = 'weights')
        conv = tf.nn.conv2d(images, kernel, [1, 4, 4, 1], padding ='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(bias, name = scope)
        print_activations(conv1)
        parameters += [kernel, biases]
        
        lrn1 = tf.nn.lrn(conv1, 4, bias=1.0, alpha=0.001/9,beta=0.75,name='lrn1')
        pool1 = tf.nn.max_pool(lrn1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool1')
        print_activations(pool1)
        
    with tf.name_scope('conv2') as scope:
        kernel = tf.Variable(tf.truncated_normal([5, 5, 64, 192],
                                                dtype = tf.float32, stddev = 1e-1), name = 'weights')
        conv = tf.nn.conv2d(pool1, kernel, [1, 1, 1, 1], padding ='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[192], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(bias, name = scope)
        print_activations(conv2)
        parameters += [kernel, biases]
        
        lrn1 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001/9,beta=0.75,name='lrn1')
        pool2 = tf.nn.max_pool(lrn1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool2')
        print_activations(pool2)
        
    with tf.name_scope('conv3') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 192, 384],
                                                dtype = tf.float32, stddev = 1e-1), name = 'weights')
        conv = tf.nn.conv2d(pool2, kernel, [1, 1, 1, 1], padding ='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[384], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv3 = tf.nn.relu(bias, name = scope)
        print_activations(conv3)
        parameters += [kernel, biases]
        
        #lrn1 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001/9,beta=0.75,name='lrn1')
        #pool1 = tf.nn.max_pool(lrn1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID', name='pool2')
        #print_activations(pool2)
    with tf.name_scope('conv4') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 384, 256],
                                                dtype = tf.float32, stddev = 1e-1), name = 'weights')
        conv = tf.nn.conv2d(conv3, kernel, [1, 1, 1, 1], padding ='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv4 = tf.nn.relu(bias, name = scope)
        print_activations(conv4)
        parameters += [kernel, biases]

    with tf.name_scope('conv5') as scope:
        kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256],
                                                dtype = tf.float32, stddev = 1e-1), name = 'weights')
        conv = tf.nn.conv2d(conv4, kernel, [1, 1, 1, 1], padding ='SAME')
        biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32), trainable=True, name='biases')
        bias = tf.nn.bias_add(conv, biases)
        conv5 = tf.nn.relu(bias, name = scope)
        print_activations(conv5)
        parameters += [kernel, biases]
        
    pool5 = tf.nn.max_pool(conv5, ksize = [1,3, 3, 1], strides = [1, 2, 2, 1],
                          padding = 'VALID', name ='pool5')
    print_activations(pool5)
    
    return pool5, parameters




In [5]:
def time_tensorflow_run(session, target, info_string):
    num_steps_burn_in = 10
    total_duration = 0.0
    total_duration_squared = 0.0
    
    for i in range(num_batches + num_steps_burn_in):
        start_time = time.time()
        _ = session.run(target)
        duration = time.time() - start_time
        if i >= num_steps_burn_in:
            if not i % 10:
                print('%s: step %d, duration =%.3f'%(
                datetime.now(), i - num_steps_burn_in, duration))
            total_duration += duration
            total_duration_squared += duration * duration
        mn = total_duration / num_batches
        vr = total_duration_squared / num_batches - mn * mn
        sd = math.sqrt(vr)
        print('%s: %s across %d steps, %.3f +/- %.3f sec / batch'%(
        datetime.now(), info_string, num_batches, mn, sd))

In [6]:
def run_benchmark():
    with tf.Graph().as_default():
        image_size = 224
        images = tf.Variable(tf.random_normal([batch_size,
                                              image_size,
                                              image_size, 3],
                                             dtype = tf.float32,
                                             stddev = 1e-1))
        pool5, parameters = inference(images)
        
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        
        time_tensorflow_run(sess, pool5, "Forward")
        
        objective = tf.nn.l2_loss(pool5)
        grad = tf.gradients(objective, parameters)
        
        time_tensorflow_run(sess, grad, "Forward-backward")

In [9]:
run_benchmark()

conv1   [32, 56, 56, 64]
conv1/pool1   [32, 27, 27, 64]
conv2   [32, 27, 27, 192]
conv2/pool2   [32, 13, 13, 192]
conv3   [32, 13, 13, 384]
conv4   [32, 13, 13, 256]
conv5   [32, 13, 13, 256]
pool5   [32, 6, 6, 256]
2017-10-27 09:16:26.395056: Forward across 100 steps, 0.000 +/- 0.000 sec / batch
2017-10-27 09:16:28.097266: Forward across 100 steps, 0.000 +/- 0.000 sec / batch
2017-10-27 09:16:29.545573: Forward across 100 steps, 0.000 +/- 0.000 sec / batch
2017-10-27 09:16:31.119061: Forward across 100 steps, 0.000 +/- 0.000 sec / batch
2017-10-27 09:16:32.575986: Forward across 100 steps, 0.000 +/- 0.000 sec / batch
2017-10-27 09:16:34.004551: Forward across 100 steps, 0.000 +/- 0.000 sec / batch
2017-10-27 09:16:35.449939: Forward across 100 steps, 0.000 +/- 0.000 sec / batch
2017-10-27 09:16:36.911792: Forward across 100 steps, 0.000 +/- 0.000 sec / batch
2017-10-27 09:16:38.366429: Forward across 100 steps, 0.000 +/- 0.000 sec / batch
2017-10-27 09:16:39.939376: Forward across 100

2017-10-27 09:18:43.545051: Forward across 100 steps, 1.236 +/- 0.580 sec / batch
2017-10-27 09:18:44.913258: Forward across 100 steps, 1.250 +/- 0.566 sec / batch
2017-10-27 09:18:46.290227: Forward across 100 steps, 1.263 +/- 0.552 sec / batch
2017-10-27 09:18:47.660036: Forward across 100 steps, 1.277 +/- 0.538 sec / batch
2017-10-27 09:18:49.031489: Forward across 100 steps, 1.291 +/- 0.522 sec / batch
2017-10-27 09:18:50.551639: Forward across 100 steps, 1.306 +/- 0.506 sec / batch
2017-10-27 09:18:52.220212: Forward across 100 steps, 1.323 +/- 0.490 sec / batch
2017-10-27 09:18:53.774994: Forward across 100 steps, 1.338 +/- 0.472 sec / batch
2017-10-27 09:18:55.302086: step 90, duration =1.527
2017-10-27 09:18:55.302232: Forward across 100 steps, 1.353 +/- 0.453 sec / batch
2017-10-27 09:18:56.802731: Forward across 100 steps, 1.368 +/- 0.432 sec / batch
2017-10-27 09:18:58.306375: Forward across 100 steps, 1.384 +/- 0.410 sec / batch
2017-10-27 09:18:59.762238: Forward across 10

2017-10-27 09:24:09.640978: step 60, duration =4.712
2017-10-27 09:24:09.641169: Forward-backward across 100 steps, 2.601 +/- 2.088 sec / batch
2017-10-27 09:24:14.424956: Forward-backward across 100 steps, 2.649 +/- 2.082 sec / batch
2017-10-27 09:24:18.820195: Forward-backward across 100 steps, 2.693 +/- 2.072 sec / batch
2017-10-27 09:24:22.959489: Forward-backward across 100 steps, 2.734 +/- 2.059 sec / batch
2017-10-27 09:24:27.111401: Forward-backward across 100 steps, 2.776 +/- 2.046 sec / batch
2017-10-27 09:24:31.478367: Forward-backward across 100 steps, 2.820 +/- 2.032 sec / batch
2017-10-27 09:24:35.814985: Forward-backward across 100 steps, 2.863 +/- 2.018 sec / batch
2017-10-27 09:24:40.186114: Forward-backward across 100 steps, 2.907 +/- 2.003 sec / batch
2017-10-27 09:24:44.585835: Forward-backward across 100 steps, 2.951 +/- 1.987 sec / batch
2017-10-27 09:24:49.036969: Forward-backward across 100 steps, 2.995 +/- 1.970 sec / batch
2017-10-27 09:24:53.996815: step 70, 